## Import packages

In [1]:
import pyomo
import pyomo.opt
import pyomo.environ as pyo
import numpy as np
import pandas as pd
import matplotlib as plt

In [2]:
n_time = 24

In [3]:
def _auxDictionary(a):
    temp_dictionary = {}
    if len(a.shape) == 3:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                for dim2 in np.arange(a.shape[2]):
                    temp_dictionary[(dim0+1, dim1+1, dim2+1)] = a[dim0, dim1, dim2]
    elif len(a.shape) == 2:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                temp_dictionary[(dim0+1, dim1+1)] = a[dim0, dim1]
    else:
        for dim0 in np.arange(a.shape[0]):
            temp_dictionary[(dim0+1)] = a[dim0]
    return temp_dictionary
#temp_dict1 = _auxDictionary(loadLimit)

## Data

In [4]:
data = {}
data['Inputs'] = pd.read_csv('Inputs.csv')
data['EVs_Inputs'] = pd.read_csv('EVs_Inputs.csv')
data['alpha'] = pd.read_csv('alpha.csv')
data['PchmaxEV'] = pd.read_csv('PchmaxEV.csv')
data['S'] = pd.read_csv('S.csv')

In [5]:
n_evs = data['EVs_Inputs']['Esoc'].size

## Start Time

In [6]:
from datetime import datetime

now = datetime.now()

start_time = now.strftime("%H:%M:%S")
print("Start Time =", start_time)

Start Time = 14:55:52


## Sets

In [7]:
model = pyo.ConcreteModel()

model.ev = pyo.Set(initialize = np.arange(1, n_evs + 1))
model.t = pyo.Set(initialize = np.arange(1, n_time + 1))


## Parameters

In [8]:
model.EEVmax = pyo.Param(model.ev, initialize =_auxDictionary(data['EVs_Inputs'].to_numpy()[:,2]))
model.EEVmin = pyo.Param(model.ev, initialize =_auxDictionary(data['EVs_Inputs'].to_numpy()[:,1]))
model.ESoc = pyo.Param(model.ev, initialize =_auxDictionary(data['EVs_Inputs'].to_numpy()[:,0]))
model.dT = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,0]))
model.cDA = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,1]))
model.Pl = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,2]))
model.Php = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,3]))
model.PchmaxEV = pyo.Param(model.ev, model.t, initialize =_auxDictionary(data['PchmaxEV'].to_numpy()))
model.S = pyo.Param(model.ev, model.t, initialize = _auxDictionary(data['S'].to_numpy()))
model.alpha = pyo.Param(model.ev, model.t, initialize = _auxDictionary(data['alpha'].to_numpy()))
model.RealHour = pyo.Param(model.t, initialize=_auxDictionary(data['Inputs'].to_numpy()[:,2]))
model.penalty1 = 1000
model.penalty2 = 1000
model.penalty3 = 0.6
model.Etrip = pyo.Param(model.ev, initialize=_auxDictionary(data['EVs_Inputs'].to_numpy()[:,3]))
model.n = 0.95
model.m = 0.6
model.Ppeak = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,3]*0.8))
model.factor = 1

## Variables

In [9]:
model.PEV = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.PEVdc = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.EEV = pyo.Var(model.ev, model.t, domain = pyo.Reals, initialize = 0)
model.Etriprelax = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.Eminsocrelax = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.Etripn = pyo.Var(model.ev, model.t, domain = pyo.Reals, initialize = 0)
model.Ppeakred = pyo.Var(model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.Ppeakred2 = pyo.Var(domain = pyo.Reals, initialize = 0)

## Constraints

In [10]:
def _balance_etripn(m,ev,t): 
    return m.Etripn[ev,t] == m.Etrip[ev]*m.S[ev,t]/(sum([m.S[ev,k] for k in np.arange(1, n_time + 1)]))
model.balance_etripn = pyo.Constraint(model.ev, model.t, rule = _balance_etripn)

def _balance_energy_EVS3(m,ev,t): 
    if t == 24:
        return m.EEV[ev,t] + m.Etriprelax[ev,t] >= m.EEVmax[ev]*m.m
    return pyo.Constraint.Skip
model.balance_energy_EVS3 = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS3)

def _balance_energy_EVS(m,ev,t): 
    if t == 1:
        return m.EEV[ev,t] - m.Etriprelax[ev,t] == m.ESoc[ev] + m.PEV[ev,t]*m.dT[t]*m.n - m.PEVdc[ev,t]*m.dT[t]/m.n - m.Etripn[ev,t]
    return pyo.Constraint.Skip
model.balance_energy_EVS = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS)

def _balance_energy_EVS2(m,ev,t): 
    if t > 1:
        return m.EEV[ev,t] - m.Etriprelax[ev,t] == m.EEV[ev,t-1] + m.PEV[ev,t]*m.dT[t]*m.n - m.PEVdc[ev,t]*m.dT[t]/m.n - m.Etripn[ev,t]
    return pyo.Constraint.Skip
model.balance_energy_EVS2 = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS2)

def _power_charging_limit1(m,ev,t): 
    return m.PEV[ev,t] >= 0
model.power_charging_limit1 = pyo.Constraint(model.ev, model.t, rule = _power_charging_limit1)

def _power_charging_limit2(m,ev,t): 
    return m.PEV[ev,t] <= m.alpha[ev,t]*m.PchmaxEV[ev,t]*m.factor
model.power_charging_limit2 = pyo.Constraint(model.ev, model.t, rule = _power_charging_limit2)

def _power_discharging_limit2(m,ev,t): 
    return m.PEVdc[ev,t] <= m.alpha[ev,t]*m.PchmaxEV[ev,t]
model.power_discharging_limit2 = pyo.Constraint(model.ev, model.t, rule = _power_discharging_limit2)

def _energy_limits_EVS_1(m,ev,t): 
    return m.EEVmin[ev] <= m.EEV[ev,t] + m.Eminsocrelax[ev,t]
    #return m.EEVmin[ev] <= m.EEV[ev,t]
model.energy_limits_EVS_1 = pyo.Constraint(model.ev, model.t, rule = _energy_limits_EVS_1)

def _energy_limits_EVS_2(m,ev,t): 
    return m.EEV[ev,t] <= m.EEVmax[ev]
model.energy_limits_EVS_2 = pyo.Constraint(model.ev, model.t, rule = _energy_limits_EVS_2)

def _power_peak_s(m,ev,t): 
    return m.Pl[t] + sum(m.PEV[e,t] - m.PEVdc[e,t] for e in np.arange(1, n_evs + 1)) - m.Ppeakred[t] <= m.Ppeak[t]    
model.power_peak_s = pyo.Constraint(model.ev, model.t, rule = _power_peak_s)

def _energy_EVs_trips(m,t): 
    return m.Ppeakred2 >= m.Pl[t] + sum(m.PEV[ev,t] - m.PEVdc[ev,t] for ev in np.arange(1, n_evs + 1))
model.energy_EVs_trips = pyo.Constraint(model.t, rule = _energy_EVs_trips)

## Objective function

In [11]:
def _FOag(m):
    return sum(m.Ppeakred[t]**2 for t in np.arange(1, n_time + 1)) + sum([m.Etriprelax[ev,t]*m.penalty1 + m.Eminsocrelax[ev,t]*m.penalty2 for ev in np.arange(1, n_evs + 1) for t in np.arange(1, n_time + 1)])

model.FOag = pyo.Objective(rule = _FOag, sense = pyo.minimize)

## Solve the model

In [12]:
from pyomo.opt import SolverFactory

model.write('res_V4_EC.lp',  io_options={'symbolic_solver_labels': True})

# Create a solver
#opt = pyo.SolverFactory('cbc', executable='C:/Program Files/Cbc-releases.2.10.8-x86_64-w64-mingw64/bin/cbc.exe')

opt = pyo.SolverFactory('cplex', executable='C:/Program Files/IBM/ILOG/CPLEX_Studio221/cplex/bin/x64_win64/cplex.exe')
opt.options['LogFile'] = 'res_V4_EC.log'

#opt = pyo.SolverFactory('ipopt', executable='C:/Program Files/Ipopt-3.11.1-win64-intel13.1/bin/ipopt.exe')
#opt.options['print_level'] = 12
#opt.options['output_file'] = "res_V5_EC.log"

results = opt.solve(model)#, tee=True)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: tmp6ybznsga
  Lower bound: 746143516.7035443
  Upper bound: 746143516.7035443
  Number of objectives: 1
  Number of constraints: 167163
  Number of variables: 124730
  Number of nonzeros: 36331323
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: 8.2
  Termination condition: optimal
  Termination message: Barrier - Optimal\x3a Objective = 7.4614351670e+08
  Error rc: 0
  Time: 35.674245834350586
# ----------------------------------------------------------
#   Solution Information
# ------------------

## Objective Function Value

In [14]:
pyo.value(model.FOag)

746143516.7034274

## End Time

In [13]:
now = datetime.now()

end_time = now.strftime("%H:%M:%S")
print("End Time =", end_time)
print("Dif: {}".format(datetime.strptime(end_time, "%H:%M:%S") - datetime.strptime(start_time, "%H:%M:%S")))

End Time = 15:05:46
Dif: 0:09:54


## Cost Value

In [15]:
def ext_pyomo_vals(vals):
    # make a pd.Series from each
    s = pd.Series(vals.extract_values(),
                  index=vals.extract_values().keys())
    # if the series is multi-indexed we need to unstack it...
    if type(s.index[0]) == tuple:    # it is multi-indexed
        s = s.unstack(level=1)
    else:
        # force transition from Series -> df
        s = pd.DataFrame(s)
    return s

In [16]:
PEV_df = ext_pyomo_vals(model.PEV)
dT_df = ext_pyomo_vals(model.dT)
cDA_df = ext_pyomo_vals(model.cDA)
PEVdc_df = ext_pyomo_vals(model.PEVdc)
EEV_df = ext_pyomo_vals(model.EEV)

charge_cost = sum([PEV_df[t][ev]*dT_df[0][t]*cDA_df[0][t]
                   for ev in np.arange(1, n_evs + 1) for t in np.arange(1, n_time + 1)])

discharge_cost = sum([PEVdc_df[t][ev]*dT_df[0][t]*cDA_df[0][t]
                      for ev in np.arange(1, n_evs + 1) for t in np.arange(1, n_time + 1)])

print('Charge cost: {}'.format(charge_cost))
print('Discharge cost: {}'.format(discharge_cost))

Charge cost: 403894.5579537475
Discharge cost: 2015717.0042277009


## Results

In [17]:
print("Total Charge: {}".format(np.sum(PEV_df.to_numpy())))

Total Charge: 14169.5097977588


In [18]:
print("Total Discharge: {}".format(np.sum(PEVdc_df.to_numpy())))

Total Discharge: 41172.02234452484


In [19]:
EEV_df

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
1,3800.0,3766.077639,3760.989468,3755.746795,3751.944503,3749.220788,3747.863389,3746.721551,3745.575097,3744.434377,...,3706.600961,3676.840961,3676.840961,3676.840961,3676.840961,3676.840961,3676.840961,3676.840961,3676.840961,4688.420481
2,2360.0,2354.324146,2349.047609,2343.622081,2339.693556,2336.885397,2335.492428,2334.320308,2333.139632,2331.964232,...,2323.613688,2318.803340,2318.803340,2318.803340,2318.803340,2318.803340,2318.803340,2318.803340,2318.803340,2929.401670
3,2360.0,2349.822608,2344.550316,2339.129001,2335.203087,2332.396523,2331.004160,2329.832509,2328.652279,2327.477258,...,2313.261620,2313.261620,2313.261620,2313.261620,2313.261620,2313.261620,2313.261620,2313.261620,2313.261620,2926.630810
4,2360.0,2281.063353,2275.793688,2270.375619,2266.454060,2263.653879,2262.267275,2261.101541,2259.926078,2258.755889,...,2107.164324,2107.164324,2107.164324,2107.164324,2107.164324,2107.164324,2107.164324,2107.164324,2107.164324,2823.582162
5,2720.0,2637.314801,2632.220280,2626.977897,2623.178594,2620.460978,2619.109462,2617.973463,2616.831453,2615.694919,...,2607.638569,2525.796388,2525.796388,2525.796388,2525.796388,2525.796388,2525.796388,2525.796388,2525.796388,3302.898194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862,2080.0,2067.514530,2062.431787,2057.203050,2053.413393,2050.703235,2049.355725,2048.223092,2047.083897,2045.949929,...,2029.578509,2029.578509,2029.578509,2029.578509,2029.578509,2029.578509,2029.578509,2029.578509,2029.578509,2574.789254
863,3800.0,3479.960000,3154.811044,3149.556307,3145.752729,3143.036986,3141.690276,3140.560048,3139.422574,3138.290706,...,2810.204922,2490.164922,2490.164922,2490.164922,2490.164922,2490.164922,2490.164922,2490.164922,2490.164922,4095.082461
864,3400.0,2755.876667,2106.816280,2101.739598,2098.056489,2095.426150,2094.121444,2093.028292,2091.925715,2090.827292,...,2082.975255,1435.586031,1435.586031,1435.586031,1435.586031,1435.586031,1435.586031,1435.586031,1435.586031,3267.793016
865,540.0,535.213718,530.574531,525.820723,522.275205,519.688855,518.372768,517.264013,516.150754,515.037851,...,506.142253,506.142253,506.142253,506.142253,506.142253,506.142253,506.142253,506.142253,506.142253,658.071126


In [20]:
PEV_df

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
1,0.0,9.298331e-08,9.382931e-08,9.383902e-08,9.239939e-08,9.123688e-08,9.411319e-08,9.554020e-08,9.598471e-08,9.589770e-08,...,9.250463e-08,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,9.423669e-08,9.518323e-08,9.516110e-08,9.359043e-08,9.228269e-08,9.510176e-08,9.648858e-08,9.693347e-08,9.683282e-08,...,9.342885e-08,9.329077e-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,9.424938e-08,9.519665e-08,9.517386e-08,9.360112e-08,9.229133e-08,9.510922e-08,9.649509e-08,9.693965e-08,9.683843e-08,...,9.343071e-08,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,9.468392e-08,9.562276e-08,9.558360e-08,9.398159e-08,9.262370e-08,9.540404e-08,9.678722e-08,9.725432e-08,9.714250e-08,...,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,9.344037e-08,9.430683e-08,9.428706e-08,9.280650e-08,9.158705e-08,9.442017e-08,9.583517e-08,9.628575e-08,9.617743e-08,...,9.269082e-08,9.256418e-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862,0.0,9.363090e-08,9.452548e-08,9.448660e-08,9.298317e-08,9.173456e-08,9.454569e-08,9.594995e-08,9.639434e-08,9.627103e-08,...,9.270442e-08,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
863,0.0,0.000000e+00,9.478135e-08,9.474822e-08,9.324379e-08,9.197636e-08,9.476905e-08,9.618911e-08,9.667640e-08,9.656086e-08,...,9.308572e-08,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
864,0.0,0.000000e+00,9.677218e-08,9.663058e-08,9.500598e-08,9.352949e-08,9.615684e-08,9.755836e-08,9.810780e-08,9.791789e-08,...,9.413930e-08,9.399923e-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
865,0.0,1.026846e-07,1.024006e-07,1.015113e-07,9.887601e-08,9.651918e-08,9.859339e-08,9.963816e-08,9.973384e-08,9.927440e-08,...,9.403777e-08,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
PEVdc_df

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
1,0.0,3.954243,4.833763,4.980540,3.612177,2.587529,1.289529,1.084746,1.089131,1.083685,...,3.341036,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,4.100061,5.012710,5.154252,3.732099,2.667751,1.323321,1.113514,1.121642,1.116630,...,3.472089,3.277831,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,4.096773,5.008677,5.150250,3.729618,2.666236,1.322745,1.113069,1.121218,1.116270,...,3.472761,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,4.091315,5.006182,5.147166,3.725481,2.660172,1.317274,1.107448,1.116689,1.111680,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,3.956939,4.839795,4.980264,3.609338,2.581736,1.283940,1.079200,1.084910,1.079707,...,3.339009,3.156072,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862,0.0,3.947697,4.828605,4.967300,3.600175,2.574650,1.280134,1.076002,1.082236,1.077269,...,3.332325,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
863,0.0,0.000000,4.853509,4.992000,3.613398,2.579956,1.279374,1.073716,1.080601,1.075275,...,3.346997,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
864,0.0,0.000000,4.690201,4.822848,3.498954,2.498822,1.239471,1.038495,1.047448,1.043502,...,3.276091,3.102596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
865,0.0,3.606468,4.407228,4.516117,3.368242,2.457033,1.250283,1.053317,1.057597,1.057258,...,3.239188,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
PEV_df.sum()

1        0.000009
2        0.000069
3      368.356902
4      606.112638
5       16.310872
6       17.416619
7       70.091877
8      926.796968
9     1504.535268
10    1811.369876
11    1963.816511
12    2012.543936
13    1806.157351
14    1310.787676
15    1745.550558
16       9.662599
17       0.000009
18       0.000009
19       0.000008
20       0.000009
21       0.000009
22       0.000009
23       0.000009
24       0.000009
dtype: float64

In [23]:
PEVdc_df.sum()

1     2125.930114
2     2876.650175
3     3643.207008
4     3749.412744
5     2728.260978
6     1957.916725
7      975.541984
8      820.247075
9      825.685375
10     822.319984
11     823.116618
12     816.374043
13     722.807458
14     646.437783
15    1969.180664
16    1263.942705
17    1864.250114
18    1945.930114
19    2023.100114
20    1846.000114
21    1685.300114
22    1733.200114
23    1632.980114
24    1674.230114
dtype: float64

In [24]:
Etriprelax_df = ext_pyomo_vals(model.Etriprelax)

In [25]:
Etriprelax_df.sum()

1     8.225044e-07
2     1.176760e+03
3     1.282896e+03
4     8.837112e+02
5     7.572326e+02
6     6.816838e+02
7     6.192837e+02
8     5.634827e+02
9     5.058990e+02
10    3.985984e+02
11    3.770377e+02
12    3.337237e+02
13    3.186621e+02
14    3.064548e+02
15    2.997005e+02
16    2.914563e+02
17    2.734888e+02
18    1.721069e+02
19    1.627061e+02
20    1.555651e+02
21    1.433953e+02
22    1.211421e+02
23    8.179160e-06
24    7.346565e+05
dtype: float64

In [26]:
Eminsocrelax_df = ext_pyomo_vals(model.Eminsocrelax)

In [27]:
Eminsocrelax_df.sum()

1     1.605819e-07
2     7.699833e-07
3     4.101385e-06
4     4.090129e-06
5     4.090127e-06
6     4.090123e-06
7     4.090116e-06
8     4.090113e-06
9     4.093805e-06
10    4.090105e-06
11    4.093683e-06
12    4.090102e-06
13    4.090113e-06
14    4.090119e-06
15    4.090160e-06
16    4.090454e-06
17    4.091733e-06
18    4.091236e-06
19    4.091148e-06
20    4.091061e-06
21    4.090986e-06
22    4.093215e-06
23    4.091832e-06
24    4.089923e-06
dtype: float64

In [28]:
Etripn_df = ext_pyomo_vals(model.Etripn)

In [29]:
Etripn_df.sum()

1         0.000000
2     99055.709983
3     54584.136167
4      7271.898119
5      3558.580983
6      2679.886649
7      2234.135530
8      1321.875530
9      1059.115952
10      351.191667
11     1321.875530
12     6730.330000
13    17814.336429
14        0.000000
15    31568.023864
16    80159.464983
17    16998.327649
18      834.574935
19      522.776364
20     4216.203167
21     8266.467333
22      799.099167
23        0.000000
24        0.000000
dtype: float64

In [30]:
Ppeakred_df = ext_pyomo_vals(model.Ppeakred)

In [31]:
Ppeakred_df

,0
1,263.157895
2,263.157894
3,263.157894
4,263.157894
5,263.157894
6,263.157894
7,263.157893
8,263.157893
9,263.157893
10,263.157893


In [32]:
Ppeakred2_df = ext_pyomo_vals(model.Ppeakred2)

In [33]:
Ppeakred2_df

,0
NaN,6552.949895


## Save results in csv files

In [34]:
import os 
folder = 'results_PeakS_b_V2G_' + str(n_evs)

if not os.path.exists(folder):
    os.makedirs(folder)
    
EEV_df.to_csv(folder + '/EEV.csv')
PEV_df.to_csv(folder + '/PEV.csv')
PEVdc_df.to_csv(folder + '/PEVdc.csv')
PEV_df.sum().to_csv(folder + '/PEV_h.csv')
PEVdc_df.sum().to_csv(folder + '/PEVdc_h.csv')

Etriprelax_df.to_csv(folder + '/Etriprelax.csv')
Etriprelax_df.sum().to_csv(folder + '/Etriprelax_h.csv')

Eminsocrelax_df.to_csv(folder + '/Eminsocrelax.csv')
Eminsocrelax_df.sum().to_csv(folder + '/Eminsocrelax_h.csv')

Etripn_df.to_csv(folder + '/Etripn.csv')
Etripn_df.sum().to_csv(folder + '/Etripn_h.csv')

Ppeakred_df.to_csv(folder + '/Ppeakread.csv')